## Библиотеки и настройки

In [1]:
# импорт библиотек

import pandas as pd # панды
pd.options.mode.chained_assignment = None  # отключаю предупреждения default='warn'
import numpy as np

import re # регулярки
import os # для работы с папками и файлами

import datetime


import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white')

import squarify 

import statistics
from scipy.stats import t
from scipy.stats import norm
from scipy import stats

from scipy.stats import pearsonr

from scipy.stats import shapiro
from scipy.stats import chi2_contingency

# для КА
from sklearn import preprocessing
from sklearn.cluster import KMeans

from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster


import io
import requests
from IPython.display import Image

%matplotlib inline

## dtp-stat

In [2]:
work_dtp = pd.read_csv('C:/00_Data/dtp-stat/dtp_stat_all.csv')

In [3]:
work_dtp.rename(columns={'datetime': 'date_time'}, inplace=True)

In [4]:
work_dtp.columns

Index(['accident_id', 'accident_type', 'category', 'region', 'district',
       'address', 'coordinates', 'type_coordinates', 'accident_lat',
       'accident_long', 'location', 'date_time', 'day_period', 'weather',
       'road_conditions', 'participants_count', 'participant_categories',
       'dead_count', 'injured_count', 'severity', 'auto_type', 'auto_brand',
       'auto_model', 'auto_year', 'auto_color', 'role', 'gender', 'violations',
       'health_status', 'years_of_driving_experience'],
      dtype='object')

In [5]:
work_dtp['violations'] = work_dtp['violations'].replace('\s+', ' ', regex=True)

In [6]:
work_dtp['violations'] = work_dtp['violations'].replace(r'\\\\xa0', ' ', regex=True)

In [7]:
work_dtp.sample(2)

,accident_id,accident_type,category,region,district,address,coordinates,type_coordinates,accident_lat,accident_long,...,auto_type,auto_brand,auto_model,auto_year,auto_color,role,gender,violations,health_status,years_of_driving_experience
1548058,2301564,['Дорожно-транспортные происшествия'],Наезд на пешехода,Псковская область,Псков,"г Псков, ул Николая Васильева, 69","{'lat': 57.8236, 'long': 28.397}",Point,57.8236,28.397,...,"В-класс (малый) до 3,9 м",ВАЗ,"ВАЗ 2110, 21101, 21102, 21103, 21108",2002.0,Серый,Водитель,Женский,[],"Раненый, находящийся (находившийся) на амбула...",none
2526371,2205335,['Дорожно-транспортные происшествия'],Наезд на пешехода,Удмуртская Республика,Ижевск,"г Ижевск, ул Спортивная, 31","{'lat': 56.0, 'long': 53.0}",Point,56.0000,53.000,...,Мотоциклы,Прочие марки мотоциклов,Прочие марки мотоциклов,2015.0,Иные цвета,Водитель,Мужской,"['Неожиданный выход из-за ТС', 'Переход через ...","Раненый, находящийся (находившийся) на стацион...",none


In [8]:
work_dtp.date_time.value_counts().head()

2015-08-05 14:35:00    62
2017-08-25 07:30:00    60
2019-12-21 09:10:00    59
2019-08-17 17:06:00    59
2021-07-09 08:50:00    58
Name: date_time, dtype: int64

In [9]:
work_dtp['year'] = work_dtp['date_time'].apply(lambda text_year: text_year[:4])

In [10]:
work_dtp['year'].value_counts()

2015    439528
2016    419953
2017    418165
2018    415311
2019    406796
2020    354274
2021    306759
Name: year, dtype: int64

In [11]:
(work_dtp.groupby(['region', 'year'], as_index=False)
 .agg({'accident_id': 'count'})
)

,region,year,accident_id
0,Алтайский край,2015,8229
1,Алтайский край,2016,7527
2,Алтайский край,2017,7447
3,Алтайский край,2018,7065
4,Алтайский край,2019,6758
...,...,...,...
590,Ярославская область,2017,4649
591,Ярославская область,2018,4820
592,Ярославская область,2019,5091
593,Ярославская область,2020,4097


In [12]:
def correct_violation_list(df):
    new_list = []
    temp_list = (df['violations']
                .replace('"', '')
                .replace('[', '')
                .replace(']', '')
                .replace(", '", ",")
                .split("',"))
    
    for i in temp_list:
        new_list.append(i.replace("'", ''))
        
        
    if new_list == ['']:
        
        new_list = ['none'] 
        
    return new_list

In [13]:
%%time

work_dtp['new_violations'] = work_dtp.apply(correct_violation_list, axis=1)

Wall time: 52.5 s


In [14]:
work_dtp.head()

,accident_id,accident_type,category,region,district,address,coordinates,type_coordinates,accident_lat,accident_long,...,auto_model,auto_year,auto_color,role,gender,violations,health_status,years_of_driving_experience,year,new_violations
0,2311491,['Дорожно-транспортные происшествия'],Наезд на препятствие,Алтайский край,Барнаул,"г Барнаул, тракт Павловский, 249 а","{'lat': 53.342, 'long': 83.6903}",Point,53.342000,83.690300,...,Camry,2002.0,Черный,Водитель,Мужской,"['Другие нарушения ПДД водителем', 'Нарушение ...",Не пострадал,NaN,2017,"[Другие нарушения ПДД водителем, Нарушение вод..."
1,2311491,['Дорожно-транспортные происшествия'],Наезд на препятствие,Алтайский край,Барнаул,"г Барнаул, тракт Павловский, 249 а","{'lat': 53.342, 'long': 83.6903}",Point,53.342000,83.690300,...,Camry,2002.0,Черный,Пассажир,Женский,['Нарушение правил применения ремней безопасно...,"Раненый, находящийся (находившийся) на амбула...",NaN,2017,[Нарушение правил применения ремней безопаснос...
2,2489516,"['Дорожно-транспортные происшествия', 'ДТП и п...",Столкновение,Алтайский край,Барнаул,"г Барнаул, ул Юрина, 241","{'lat': 53.369248, 'long': 83.699802}",Point,53.369248,83.699802,...,Forester,2006.0,Серый,Водитель,Мужской,['Несоответствие скорости конкретным условиям ...,"Раненый, находящийся (находившийся) на амбулат...",12,2020,[Несоответствие скорости конкретным условиям д...
3,2489516,"['Дорожно-транспортные происшествия', 'ДТП и п...",Столкновение,Алтайский край,Барнаул,"г Барнаул, ул Юрина, 241","{'lat': 53.369248, 'long': 83.699802}",Point,53.369248,83.699802,...,Forester,2006.0,Серый,Пассажир,Женский,[],"Раненый, находящийся (находившийся) на амбулат...",NaN,2020,[none]
4,2489516,"['Дорожно-транспортные происшествия', 'ДТП и п...",Столкновение,Алтайский край,Барнаул,"г Барнаул, ул Юрина, 241","{'lat': 53.369248, 'long': 83.699802}",Point,53.369248,83.699802,...,Pajero,2014.0,Белый,Водитель,Мужской,[],Не пострадал,29,2020,[none]


In [15]:
work_dtp['new_violations'][3]

['none']

In [16]:
type(work_dtp['new_violations'][3])

list

In [18]:
# список нарушений

data_violations = []

for violation_list in work_dtp['new_violations']:
    for violation in violation_list:
        data_violations.append(violation)

In [20]:
data_violations = list(set(data_violations))

In [21]:
data_violations.sort()

In [22]:
len(data_violations)

112

In [23]:
# словарь для подсчета

violation_dict = {}

for i in data_violations:
    violation_dict[i] = 0

In [24]:
%%time

# цикл для подсчета значений

for item in data_violations:
    
    for violations_item in work_dtp['new_violations']:
        
        for one_violation in violations_item:
            if item == one_violation:
                violation_dict[item] += 1

Wall time: 3min 31s


In [25]:
work_violations = pd.DataFrame().from_dict(violation_dict, orient='index').reset_index()

In [26]:
work_violations.columns=['violation', 'number']

In [27]:
work_violations.head(7)

,violation,number
0,none,1424171
1,Выезд на полосу встречного движения,85115
2,"Выезд на полосу встречного движения в местах, ...",11792
3,Выезд на полосу встречного движения с разворот...,5581
4,Выезд на трамвайные пути встречного направления,54
...,...,...
107,Ходьба вдоль проезжей части попутного направле...,2756
108,Ходьба вдоль проезжей части при наличии и удов...,2087
109,Эксплуатация ТС с техническими неисправностями...,4994
110,Эксплуатация незарегистрированного ТС,22174


In [ ]:
#work_violations.to_excel('C:/00_Projects/RIA/2022/02_elk_accidents/00_data/all_dtp.xlsx')

## ДТП с животными

In [28]:
work_dtp.columns

Index(['accident_id', 'accident_type', 'category', 'region', 'district',
       'address', 'coordinates', 'type_coordinates', 'accident_lat',
       'accident_long', 'location', 'date_time', 'day_period', 'weather',
       'road_conditions', 'participants_count', 'participant_categories',
       'dead_count', 'injured_count', 'severity', 'auto_type', 'auto_brand',
       'auto_model', 'auto_year', 'auto_color', 'role', 'gender', 'violations',
       'health_status', 'years_of_driving_experience', 'year',
       'new_violations'],
      dtype='object')

In [29]:
work_dtp['category'].value_counts().reset_index()

,index,category
0,Столкновение,1452653
1,Наезд на пешехода,672599
2,Опрокидывание,141775
3,Наезд на препятствие,125329
4,Съезд с дороги,100349
5,Наезд на стоящее ТС,96029
6,Падение пассажира,76143
7,Наезд на велосипедиста,74000
8,Иной вид ДТП,9402
9,Наезд на животное,6540


In [30]:
work_animals = work_dtp[work_dtp['category'] == 'Наезд на животное']

In [31]:
work_animals.shape

(6540, 32)

In [32]:
# словарь для подсчета

violation_dict_animal = {}

for i in data_violations:
    violation_dict_animal[i] = 0

In [33]:
%%time

# цикл для подсчета значений

for item in violation_dict_animal:
    
    for violations_item in work_animals['new_violations']:
        
        for one_violation in violations_item:
            if item == one_violation:
                violation_dict_animal[item] += 1

Wall time: 761 ms


In [34]:
work_violations_animal = pd.DataFrame().from_dict(violation_dict_animal, orient='index').reset_index()

In [35]:
work_violations_animal.columns=['violation', 'number_animal']

In [36]:
work_violations_animal

,violation,number_animal
0,none,3449
1,Выезд на полосу встречного движения,28
2,"Выезд на полосу встречного движения в местах, ...",3
3,Выезд на полосу встречного движения с разворот...,2
4,Выезд на трамвайные пути встречного направления,0
...,...,...
107,Ходьба вдоль проезжей части попутного направле...,0
108,Ходьба вдоль проезжей части при наличии и удов...,0
109,Эксплуатация ТС с техническими неисправностями...,7
110,Эксплуатация незарегистрированного ТС,88


In [37]:
work_animals.to_excel('C:/00_Projects/RIA/2022/02_elk_accidents/00_data/animal_dtp.xlsx', index=False, sheet_name='original')

## Дикие животные

In [38]:
work_animals_wild = pd.read_excel('C:/00_Projects/RIA/2022/02_elk_accidents/00_data/animal_dtp.xlsx', sheet_name='data')

In [ ]:
(work_animals[work_animals.dead_count > 0]
 .groupby('auto_brand', as_index=False)
 .agg({'accident_id': 'count'})
 .sort_values('accident_id', ascending=False)
).head(10)